## 下载环境

pip install transformers
pip install modelscope
pip install datasets
pip install accelerate
pip install bitsandbytes 
pip install peft
pip install swanlab
pip install sentencepiece
pip install trl 


pip install unsloth

## 下载数据集

In [1]:
import subprocess
import os

result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value

from huggingface_hub import notebook_login

notebook_login()

In [2]:
from datasets import load_dataset


ds_reason = load_dataset("Ronndy/medical_o1_sft_Chinese",cache_dir='./data/reason')
ds_no_reason = load_dataset("BAAI/IndustryInstruction_Health-Medicine",cache_dir = './data/no_reason')

Using the latest cached version of the dataset since Ronndy/medical_o1_sft_Chinese couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at data/reason/Ronndy___medical_o1_sft_chinese/default/0.0.0/b9f185ad9b04dc86473b9798513089d6be910d4d (last modified on Mon May 12 18:41:33 2025).
Using the latest cached version of the dataset since BAAI/IndustryInstruction_Health-Medicine couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at data/no_reason/BAAI___industry_instruction_health-medicine/default/0.0.0/010a7df17056ca13601ea15dfb7d753a4aac1efe (last modified on Mon May 12 16:54:20 2025).


In [3]:
ds_reason

DatasetDict({
    train: Dataset({
        features: ['messages'],
        num_rows: 17340
    })
    test: Dataset({
        features: ['messages'],
        num_rows: 3717
    })
})

In [4]:
ds_reason['train'][0]

{'messages': [{'content': '你是一个医生，能够根据病人的表述为病人诊断病情，当然你也有跟人交流普通问题的情况。',
   'role': 'system'},
  {'content': '一刚出生新生儿,身体红,四肢青紫,心率90次/分,呼吸20次/分,不规则,四肢略屈曲,弹足底有皱眉。该新生儿Apgar评分为\nA. 8分\nB. 7分\nC. 6分\nD. 5分\nE. 4分',
   'role': 'user'},
  {'content': '<think>\n嗯，这个新生儿刚出生身体是红色的，但是四肢是青紫色的。看上去身体的血液循环似乎还不错，但是四肢稍微差一点。躯干部位在Apgar评分中可以算2分，四肢青紫得不了分，所以颜色这部分我认为总共可以算1分。\n\n哦，他的心率是90次每分钟。根据Apgar评分的标准，心率要超过100才可以得2分。90次不到100次但还不是最糟糕的情况下，可以算1分。\n\n然后是呼吸情况，宝宝的呼吸好像有点不规则。对于Apgar评分来说，不规则呼吸大概算是1分吧，而正常呼吸是2分的标准。这样看来，呼吸部分只得1分。\n\n肌张力方面，小家伙四肢略微屈曲。完全屈曲是最好的，得2分，但他只是稍微屈曲，所以这里给1分妥当。\n\n哦对，反射兴奋性这个也要看一下。他在弹足底时有皱眉的反应。这个皱眉呢，在评分中算是轻度反应，得1分。要是能有更明显的反应比如哭，那才能得2分。\n\n综合来看，这个小宝宝在评分上颜色1分，心率1分，呼吸1分，肌张力1分，反射兴奋性1分，总共加起来应该是5分。果然是D，没错。\n</think>\n<response>根据Apgar评分标准：\n\n1. 颜色：身体红、四肢青紫，计1分。\n2. 心率：90次/分，计1分。\n3. 呼吸：不规则，计1分。\n4. 肌张力：四肢略屈曲，计1分。\n5. 反射兴奋性：弹足底有皱眉，计1分。\n\n综合以上五项指标，该新生儿的Apgar评分为5分。因此，正确选项是D. 5分。</response>',
   'role': 'assistant'}]}

In [8]:
def convert_to_alpaca_format(examples):
    conversations = []
    for messages in examples["messages"]:
        # 提取系统提示词
        system_prompt = next((msg["content"] for msg in messages if msg["role"] == "system"), "")
        
        # 提取用户问题和助手回答
        user_question = next((msg["content"] for msg in messages if msg["role"] == "user"), "")
        assistant_answer = next((msg["content"] for msg in messages if msg["role"] == "assistant"), "")
        
        # 分割think和response部分
        think_part = ""
        response_part = ""
        if assistant_answer:
            think_start = assistant_answer.find("<think>")
            think_end = assistant_answer.find("</think>")
            if think_start != -1 and think_end != -1:
                think_part = assistant_answer[think_start+7:think_end].strip()
            
            response_start = assistant_answer.find("<response>")
            response_end = assistant_answer.find("</response>")
            if response_start != -1 and response_end != -1:
                response_part = assistant_answer[response_start+10:response_end].strip()
            else:
                # 如果没有response标签，尝试获取think标签之后的内容
                response_part = assistant_answer[think_end+8:].strip()
        
        # 构建Alpaca格式的对话
        conversation = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_question},
            {"role": "assistant", "content": f'<think>{think_part}</think>{response_part}'}
        ]
        conversations.append(conversation)
    
    return {"conversations": conversations}

# 批量处理数据集
ds_reason_alpaca = ds_reason['train'].map(
    convert_to_alpaca_format,
    batched=True,
    remove_columns=ds_reason["train"].column_names,
)

In [10]:
ds_reason_alpaca[0]

{'conversations': [{'content': '你是一个医生，能够根据病人的表述为病人诊断病情，当然你也有跟人交流普通问题的情况。',
   'role': 'system'},
  {'content': '一刚出生新生儿,身体红,四肢青紫,心率90次/分,呼吸20次/分,不规则,四肢略屈曲,弹足底有皱眉。该新生儿Apgar评分为\nA. 8分\nB. 7分\nC. 6分\nD. 5分\nE. 4分',
   'role': 'user'},
  {'content': '<think>嗯，这个新生儿刚出生身体是红色的，但是四肢是青紫色的。看上去身体的血液循环似乎还不错，但是四肢稍微差一点。躯干部位在Apgar评分中可以算2分，四肢青紫得不了分，所以颜色这部分我认为总共可以算1分。\n\n哦，他的心率是90次每分钟。根据Apgar评分的标准，心率要超过100才可以得2分。90次不到100次但还不是最糟糕的情况下，可以算1分。\n\n然后是呼吸情况，宝宝的呼吸好像有点不规则。对于Apgar评分来说，不规则呼吸大概算是1分吧，而正常呼吸是2分的标准。这样看来，呼吸部分只得1分。\n\n肌张力方面，小家伙四肢略微屈曲。完全屈曲是最好的，得2分，但他只是稍微屈曲，所以这里给1分妥当。\n\n哦对，反射兴奋性这个也要看一下。他在弹足底时有皱眉的反应。这个皱眉呢，在评分中算是轻度反应，得1分。要是能有更明显的反应比如哭，那才能得2分。\n\n综合来看，这个小宝宝在评分上颜色1分，心率1分，呼吸1分，肌张力1分，反射兴奋性1分，总共加起来应该是5分。果然是D，没错。</think>根据Apgar评分标准：\n\n1. 颜色：身体红、四肢青紫，计1分。\n2. 心率：90次/分，计1分。\n3. 呼吸：不规则，计1分。\n4. 肌张力：四肢略屈曲，计1分。\n5. 反射兴奋性：弹足底有皱眉，计1分。\n\n综合以上五项指标，该新生儿的Apgar评分为5分。因此，正确选项是D. 5分。',
   'role': 'assistant'}]}

## 注意事项